## Classification
Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

In [1]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [2]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Get the data first

In [3]:
iris = dataset("datasets", "iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [4]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [5]:
X

150×4 Array{Float64,2}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 5.4  3.7  1.5  0.2
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 ⋮              
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [6]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

150-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [7]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [8]:
?randsubseq

search: randsubseq randsubseq! RandomSub StratifiedRandomSub



```
randsubseq([rng=GLOBAL_RNG,] A, p) -> Vector
```

Return a vector consisting of a random subsequence of the given array `A`, where each element of `A` is included (in order) with independent probability `p`. (Complexity is linear in `p*length(A)`, so this function is efficient even if `p` is small and `A` is large.) Technically, this process is known as "Bernoulli sampling" of `A`.

# Examples

```jldoctest
julia> rng = MersenneTwister(1234);

julia> randsubseq(rng, collect(1:8), 0.3)
2-element Array{Int64,1}:
 7
 8
```


In [9]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

51-element Array{Int64,1}:
   2
   4
  10
  12
  13
  17
  21
  22
  23
  25
  26
  27
  28
   ⋮
 111
 124
 125
 126
 127
 129
 133
 136
 140
 144
 146
 148

We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous.

In [10]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 🟣 Method 1: Lasso

In [11]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
71 models for 4 predictors in 10 folds
Best λ 0.001 (mean loss 0.057, std 0.005)

In [12]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [13]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

51×1 Array{Float64,2}:
 1.0137504950307894
 1.0148017024933282
 0.9520491218707511
 0.9866846373551885
 0.9509979144082121
 0.9923630858100112
 1.0023199811180667
 1.0528020459364464
 0.8637043021772398
 1.0335906686438234
 1.045021182556546
 1.112189798600343
 0.9564651572919708
 ⋮
 2.692636392862999
 2.608781225593785
 2.8346168997625067
 2.7078493254837492
 2.578561745530567
 2.8919022375013252
 2.9546548181239025
 3.0664158449601926
 2.8168791410745504
 3.005976884833757
 2.9256977510942876
 2.737440009226556

In [14]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

1.0

### 🟣 Method 2: Ridge
We will use the same function but set alpha to zero.

In [15]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

1.0

### 🟣 Method 3: Elastic Net
We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

In [16]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

1.0

### 🟣 Method 4: Decision Trees
We will use the package `DecisionTree`

In [17]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [18]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.9607843137254902

### 🟣 Method 5: Random Forests
The `RandomForestClassifier` is available through the `DecisionTree` package as well.

In [19]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 6.15
Avg Depth:  4.2

In [20]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9803921568627451

### 🟣 Method 6: Using a Nearest Neighbor method
We will use the `NearestNeighbors` package here.

In [21]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SArray{Tuple{4},Float64,1,4},Euclidean,Float64}
  Number of points: 99
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [22]:
queries = X[testids,:]

51×4 Array{Float64,2}:
 4.9  3.0  1.4  0.2
 4.6  3.1  1.5  0.2
 4.9  3.1  1.5  0.1
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 5.4  3.9  1.3  0.4
 5.4  3.4  1.7  0.2
 5.1  3.7  1.5  0.4
 4.6  3.6  1.0  0.2
 4.8  3.4  1.9  0.2
 5.0  3.0  1.6  0.2
 5.0  3.4  1.6  0.4
 5.2  3.5  1.5  0.2
 ⋮              
 6.5  3.2  5.1  2.0
 6.3  2.7  4.9  1.8
 6.7  3.3  5.7  2.1
 7.2  3.2  6.0  1.8
 6.2  2.8  4.8  1.8
 6.4  2.8  5.6  2.1
 6.4  2.8  5.6  2.2
 7.7  3.0  6.1  2.3
 6.9  3.1  5.4  2.1
 6.8  3.2  5.9  2.3
 6.7  3.0  5.2  2.3
 6.5  3.0  5.2  2.0

In [23]:
idxs, dists = knn(kdtree, queries', 5, true)

([[20, 27, 18, 21, 2], [28, 17, 18, 2, 27], [20, 18, 30, 17, 27], [17, 6, 18, 5, 30], [27, 20, 18, 2, 28], [8, 29, 14, 4, 10], [19, 16, 8, 23, 29], [14, 12, 3, 29, 1], [5, 2, 22, 24, 28], [17, 18, 6, 15, 23]  …  [81, 93, 73, 67, 96], [85, 86, 81, 73, 68], [84, 92, 97, 53, 88], [67, 72, 77, 91, 73], [67, 72, 73, 93, 77], [86, 68, 83, 71, 79], [73, 94, 81, 93, 77], [81, 96, 93, 67, 73], [94, 73, 76, 93, 81], [72, 77, 76, 91, 49]], [[0.14142135623730964, 0.14142135623730986, 0.24494897427831822, 0.3, 0.30000000000000016], [0.14142135623730964, 0.17320508075688815, 0.22360679774997916, 0.24494897427831802, 0.26457513110645925], [0.1, 0.1732050807568881, 0.26457513110645875, 0.26457513110645925, 0.2645751311064593], [0.22360679774997858, 0.22360679774997916, 0.2999999999999998, 0.3000000000000002, 0.30000000000000027], [0.19999999999999998, 0.20000000000000037, 0.244948974278318, 0.26457513110645897, 0.3000000000000002], [0.3464101615137753, 0.3605551275463989, 0.38729833462074226, 0.399999

In [24]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(argmax(DataFrame(possible_labels[i])[1,:]))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

1.0

### 🟣 Method 7: Support Vector Machines
We will use the `LIBSVM` package here.

In [25]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

99-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [26]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Int64,Float64}(36, Int32[4, 16, 16], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.4 4.3 … 6.3 5.9; 2.9 3.0 … 2.5 3.0; 1.4 1.1 … 5.0 5.1; 0.2 0.1 … 1.9 1.8], Int32[7, 9, 11, 15, 31, 33, 34, 35, 38, 41  …  84, 85, 87, 88, 89, 92, 94, 95, 97, 99], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.4), LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 7.0), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.0), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 5.6)  …  LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.6758338451531314 0.4655767891335155; 0.0 0.46483582345841046; … ; -0.0 -1.0; -0.0 -1.0], F

In [27]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

1.0

Putting all the results together:

In [28]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Array{Any,2}:
 "lasso"  1.0
 "ridge"  1.0
 "EN"     1.0
 "DT"     0.960784
 "RF"     0.980392
 "kNN"    1.0
 "SVM"    1.0

# Finally...
After finishing this notebook, you should be able to:
- [ ] split your data into training and testing data to test the effectiveness of a certain method
- [ ] apply a simple accuracy function to test the effectiveness of a certain method
- [ ] run multiple classification algorithms:
    - [ ] LASSO
    - [ ] Ridge
    - [ ] ElasticNet
    - [ ] Decision Tree
    - [ ] Random Forest
    - [ ] Nearest Neighbors
    - [ ] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |1.0|
| ridge  |1.0|
| EN     |1.0|
| DT     |0.960784|
| RF     |0.980392|
| kNN    |1.0|
| SVM    |1.0|